![Django Ledger](../django_ledger/static/django_ledger/logo_2/django_ledger_logo_dark@2x.png "Django Ledger Logo")

In [1]:
import os
from datetime import date, datetime
from random import randint, choices, random
from zoneinfo import ZoneInfo

import django
# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependency of django_ledger...
from django.core.exceptions import ObjectDoesNotExist

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'dev_env.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
# os.chdir('../')

django.setup()

from django_ledger.models.entity import EntityModel
from django_ledger.models.items import ItemModel
from django_ledger.models.invoice import InvoiceModel
from django_ledger.models.bill import BillModel
from django_ledger.models.estimate import EstimateModel
from django.contrib.auth import get_user_model
from django_ledger.io import roles
from django_ledger.io.io_library import IOBluePrint, IOLibrary

try:
    import pandas as pd

    PANDAS_INSTALLED = True
except ImportError:
    PANDAS_INSTALLED = False

# Get Your Entity Administrator UserModel

In [2]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    user_model = UserModel.objects.get(username__exact=MY_USERNAME)
except ObjectDoesNotExist:
    user_model = UserModel(username=MY_USERNAME)
    user_model.set_password(MY_PASSWORD)
    user_model.save()

# Get or Create an Entity Model

In [3]:
ENTITY_NAME = 'One Big Company, LLC'

entity_model = EntityModel.create_entity(
    name=ENTITY_NAME,
    admin=user_model,
    use_accrual_method=True,
    fy_start_month=1
)

entity_model

<EntityModel: EntityModel one-big-company-llc-8s39adzs: One Big Company, LLC>

# Chart of Accounts

## Create a Default Chart of Accounts
- Newly created EntityModel do not have a default Code of Accounts yet.
- Django Ledger support multiple chart of accounts, but only one can be assigned as default.

### Check if entity has a default CoA

In [4]:
entity_model.has_default_coa()

False

In [5]:
default_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=True,
    commit=True,
    coa_name='My QuickStart CoA'
)

In [6]:
default_coa_model

<ChartOfAccountModel: My QuickStart CoA (coa-9adzs-a7sjgs81fkl9sm4)>

### EntityModel has now a Default Chart of Accounts

In [7]:
entity_model.has_default_coa()

True

In [8]:
default_coa_model = entity_model.get_default_coa()
default_coa_model

<ChartOfAccountModel: My QuickStart CoA (coa-9adzs-a7sjgs81fkl9sm4)>

### Default Chart of Accounts is accessible from the Entity Model

In [9]:
entity_model.default_coa == default_coa_model

True

# Populate Entity with Random Data (Optional)

If you are getting started with Django Ledger, you may want to populate an entity with random data to help you get familiar with the API.

### Define a Start Date for Transactions

In [10]:
START_DTTM = datetime(year=2022, month=10, day=1, tzinfo=ZoneInfo('UTC'))

### Fill the entity with random data.
- This action will populate the EntityModel with random data.
- It will populate a Code of Accounts using a default pre-defined list.
- This approach is for illustration, educational and testing purposes, not encouraged for new production entities.
- Only Entities with no transactions can use this method.

In [11]:
entity_model.populate_random_data(start_date=START_DTTM)

# Chart of Accounts (CoA)
- A Chart of Accounts is a user-defined list of accounts. 
- Each Entity Model must have at least one default Chart of Accounts.

## Django Ledger supports multiple chart of accounts.

In [12]:
another_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=False,
    commit=True,
    coa_name='My Empty Chart of Accounts'
)

In [13]:
another_coa_model

<ChartOfAccountModel: My Empty Chart of Accounts (coa-9adzs-gim7may1bdymr55)>

# Accounts

## Default CoA Accounts
- These accounts are automatically created when populating the Entity with random data.

In [14]:
default_coa_accounts_qs = entity_model.get_default_coa_accounts().values('code', 'name', 'role', 'balance_type',
                                                                         'active', 'locked')
pd.DataFrame(default_coa_accounts_qs) if PANDAS_INSTALLED else default_coa_accounts_qs


<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1100', 'name': 'Accounts Receivable', 'role': 'asset_ca_recv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1110', 'name': 'Uncollectibles', 'role': 'asset_ca_uncoll', 'balance_type': 'credit', 'active': True, 'locked': False}, {'code': '1200', 'name': 'Inventory', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1300', 'name': 'Prepaid Expenses', 'role': 'asset_ca_prepaid', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1510', 'name': 'Notes Receivable', 'role': 'asset_lti_notes', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1520', 'name': 'Land', 'role': 'asset_lti_land', 'balance_type': '

## Get CoA Accounts by Explicit CoA Model (In case of multiple CoAs)

In [15]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=default_coa_model).values('code', 'name', 'role',
                                                                                                 'balance_type',
                                                                                                 'active', 'locked')
pd.DataFrame(coa_accounts_by_coa_model_qs) if PANDAS_INSTALLED else coa_accounts_by_coa_model_qs

<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1100', 'name': 'Accounts Receivable', 'role': 'asset_ca_recv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1110', 'name': 'Uncollectibles', 'role': 'asset_ca_uncoll', 'balance_type': 'credit', 'active': True, 'locked': False}, {'code': '1200', 'name': 'Inventory', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1300', 'name': 'Prepaid Expenses', 'role': 'asset_ca_prepaid', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1510', 'name': 'Notes Receivable', 'role': 'asset_lti_notes', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1520', 'name': 'Land', 'role': 'asset_lti_land', 'balance_type': '

No Accounts yet on this CoA...

In [16]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=another_coa_model).values('code', 'name', 'role',
                                                                                                 'balance_type',
                                                                                                 'active', 'locked')
pd.DataFrame(coa_accounts_by_coa_model_qs) if PANDAS_INSTALLED else coa_accounts_by_coa_model_qs

<AccountModelQuerySet []>

## Get CoA Accounts by CoA Model UUID
- May pass UUID instance instead of ChartOF AccountsModel.
- Useful when using the CoA model UUID as URL kwarg.

In [17]:
coa_accounts_by_coa_uuid_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.uuid).values('code', 'name',
                                                                                                     'role',
                                                                                                     'balance_type',
                                                                                                     'active', 'locked')
pd.DataFrame(coa_accounts_by_coa_uuid_qs) if PANDAS_INSTALLED else coa_accounts_by_coa_uuid_qs

<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1100', 'name': 'Accounts Receivable', 'role': 'asset_ca_recv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1110', 'name': 'Uncollectibles', 'role': 'asset_ca_uncoll', 'balance_type': 'credit', 'active': True, 'locked': False}, {'code': '1200', 'name': 'Inventory', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1300', 'name': 'Prepaid Expenses', 'role': 'asset_ca_prepaid', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1510', 'name': 'Notes Receivable', 'role': 'asset_lti_notes', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1520', 'name': 'Land', 'role': 'asset_lti_land', 'balance_type': '

## Get CoA Accounts by CoA Model Slug
- If string is passed, will lookup by slug.
- Useful when using the CoA model UUID as URL kwarg.

In [18]:
coa_accounts_by_coa_slug_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.slug).values('code', 'name',
                                                                                                     'role',
                                                                                                     'balance_type',
                                                                                                     'active', 'locked')
pd.DataFrame(coa_accounts_by_coa_slug_qs) if PANDAS_INSTALLED else coa_accounts_by_coa_slug_qs

<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1100', 'name': 'Accounts Receivable', 'role': 'asset_ca_recv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1110', 'name': 'Uncollectibles', 'role': 'asset_ca_uncoll', 'balance_type': 'credit', 'active': True, 'locked': False}, {'code': '1200', 'name': 'Inventory', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1300', 'name': 'Prepaid Expenses', 'role': 'asset_ca_prepaid', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1510', 'name': 'Notes Receivable', 'role': 'asset_lti_notes', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1520', 'name': 'Land', 'role': 'asset_lti_land', 'balance_type': '

## Get Accounts With Codes and CoA Model
- Assumes default CoA if no coa_model is passed...

In [19]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(code_list=['1010', '1050']).values('code', 'name',
                                                                                                   'role',
                                                                                                   'balance_type',
                                                                                                   'active', 'locked')
pd.DataFrame(coa_accounts_by_codes_qs) if PANDAS_INSTALLED else coa_accounts_by_codes_qs

<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}]>

Empty ChartOfAccountModel...

In [20]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(
    code_list=['1010', '1050'],
    coa_model=another_coa_model
).values('code', 'name', 'role', 'balance_type', 'active', 'locked')
pd.DataFrame(coa_accounts_by_codes_qs) if PANDAS_INSTALLED else coa_accounts_by_codes_qs

<AccountModelQuerySet []>

### Get All Accounts at Once
- Convenience method to get all accounts from all chart of account models associated with the entity.

In [21]:
coa_qs, coa_map = entity_model.get_all_coa_accounts()

A dictionary, CoA Model -> Account List.

In [22]:
coa_map.keys()

dict_keys([<ChartOfAccountModel: My QuickStart CoA (coa-9adzs-a7sjgs81fkl9sm4)>, <ChartOfAccountModel: My Empty Chart of Accounts (coa-9adzs-gim7may1bdymr55)>])

In [23]:
coa_map[default_coa_model].values('code', 'name', 'role', 'balance_type', 'active', 'locked')

<AccountModelQuerySet [{'code': '1010', 'name': 'Cash', 'role': 'asset_ca_cash', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1050', 'name': 'Short Term Investments', 'role': 'asset_ca_mkt_sec', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1100', 'name': 'Accounts Receivable', 'role': 'asset_ca_recv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1110', 'name': 'Uncollectibles', 'role': 'asset_ca_uncoll', 'balance_type': 'credit', 'active': True, 'locked': False}, {'code': '1200', 'name': 'Inventory', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1300', 'name': 'Prepaid Expenses', 'role': 'asset_ca_prepaid', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1510', 'name': 'Notes Receivable', 'role': 'asset_lti_notes', 'balance_type': 'debit', 'active': True, 'locked': False}, {'code': '1520', 'name': 'Land', 'role': 'asset_lti_land', 'balance_type': '

In [24]:
coa_map[another_coa_model].values('code', 'name', 'role', 'balance_type', 'active', 'locked')

<AccountModelQuerySet []>

## Create Account Model
- Creating AccountModel into empty "another_coa_model"...

In [25]:
account_model = entity_model.create_account(
    coa_model=another_coa_model,
    code=f'1{str(randint(10000, 99999))}',
    role=roles.ASSET_CA_INVENTORY,
    name='A cool account created from the EntityModel API!',
    balance_type=roles.DEBIT,
    active=True
)

In [26]:
account_model

<AccountModel: ASSETS - 118965: A cool account created from the EntityModel API! (ASSET_CA_INV/debit)>

In [27]:
another_coa_accounts_qs = entity_model.get_coa_accounts(coa_model=another_coa_model).values('code', 'name', 'role',
                                                                                            'balance_type', 'active',
                                                                                            'locked')
pd.DataFrame(another_coa_accounts_qs) if PANDAS_INSTALLED else another_coa_accounts_qs

<AccountModelQuerySet [{'code': '118965', 'name': 'A cool account created from the EntityModel API!', 'role': 'asset_ca_inv', 'balance_type': 'debit', 'active': True, 'locked': False}]>

# Basic Django Ledger Usage
- The LedgerModel enables object-oriented programming into the accounting system.
- Examples:
    - A month.
    - A customer.
    - A vendor.
    - A project.
    - A building.
- The more appropriate ledgers are created, the more segregation and control over transactions is possible.

## Creating an Empty Ledger using the Entity API.

In [28]:
ledger_model = entity_model.create_ledger(name='My October 2023 Ledger', posted=True)

## Creating Transaction Libraries

In [29]:
library = IOLibrary(name='quickstart-library')

## Create and Register a BluePrint

In [30]:
@library.register
def sale_blueprint(
        sale_amount,
        contribution_margin_percent: float,
        description: str = None
) -> IOBluePrint:
    blueprint = IOBluePrint()
    cogs_amount = (1 - contribution_margin_percent) * sale_amount
    blueprint.debit(account_code='1010', amount=sale_amount, description=description)
    blueprint.credit(account_code='4010', amount=sale_amount, description=description)
    blueprint.credit(account_code='1200', amount=cogs_amount, description=description)
    blueprint.debit(account_code='5010', amount=cogs_amount, description=description)
    return blueprint

## Get a Cursor

In [31]:
cursor = library.get_cursor(entity_model=entity_model, user_model=user_model)

## Dispatch Some Instructions

In [32]:
cursor.dispatch('sale_blueprint',
                ledger_model='ledger-test-1',
                sale_amount=120.345,
                contribution_margin_percent=0.25,
                description='so cool')
cursor.dispatch('sale_blueprint',
                ledger_model='ledger-test-1',
                sale_amount=12.345,
                contribution_margin_percent=0.2,
                description='so cool')
cursor.dispatch('sale_blueprint',
                ledger_model=ledger_model,
                sale_amount=34.455,
                contribution_margin_percent=0.13,
                description='so cool')
cursor.dispatch('sale_blueprint',
                ledger_model='ledger-test-12',
                sale_amount=90.43,
                contribution_margin_percent=0.17,
                description='so cool')

## Commit Your Instructions
Not recommended to post both ledger and journal entries. Posted transactions will immediately affect the books.
**result** contains the resulting ledger models, journal entries and transactions. 

In [33]:
result = cursor.commit(
    post_new_ledgers=True,
    post_journal_entries=True,
    je_timestamp='2023-12-02 12:00'
)

In [34]:
result

{<LedgerModel: LedgerModel: Blueprint Commitment ledger-test-1 | Posted: True | Locked: False>: {'ledger_model': <LedgerModel: LedgerModel: Blueprint Commitment ledger-test-1 | Posted: True | Locked: False>,
  'journal_entry': <JournalEntryModel: JE: JE-2023-000-0000000019 (posted=True, locked=True) - Desc: >,
  'txs_models': [<TransactionModel: 1010-Cash/debit: 120.34/debit>,
   <TransactionModel: 4010-Sales Income/credit: 120.34/credit>,
   <TransactionModel: 1200-Inventory/debit: 90.26/credit>,
   <TransactionModel: 5010-Cost of Goods Sold/debit: 90.26/debit>,
   <TransactionModel: 1010-Cash/debit: 12.35/debit>,
   <TransactionModel: 4010-Sales Income/credit: 12.35/credit>,
   <TransactionModel: 1200-Inventory/debit: 9.88/credit>,
   <TransactionModel: 5010-Cost of Goods Sold/debit: 9.88/debit>],
  'instructions': [TransactionInstructionItem(account_code='1010', amount=Decimal('120.34'), tx_type='debit', description='so cool', account_model=<AccountModel: ASSETS - 1010: Cash (ASSET_

### Get Financial Statement Report Data fro Ledger Model

Balance Sheet

In [35]:
bs_data = ledger_model.digest_balance_sheet(
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

bs_data.get_balance_sheet_data()

{'assets': {'total_balance': Decimal('4.4700000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('ceca82da-05a2-4f09-8033-e1da9508e111'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('34.4500000000000'),
      'balance_abs': Decimal('34.4500000000000')}],
    'total_balance': Decimal('34.4500000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_inv': {'accounts': [{'account_uuid': UUID('213e4ae3-2603-4957-ac28-4596dff79716'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
   

Income Statement

In [36]:
is_data = ledger_model.digest_income_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)
is_data.get_income_statement_data()

{'operating': {'revenues': [{'account_uuid': UUID('13d064e4-365c-46aa-b480-2a4c8f5a79ae'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('34.4500000000000'),
    'balance_abs': Decimal('34.4500000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('c6705d64-4c39-483c-9be8-9dd989a9db0e'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-29.9800000000000'),
    'balance_abs':

Cash Flow Statement

In [37]:
cfs_data = ledger_model.digest_cash_flow_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

cfs_data.get_cash_flow_statement_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('4.4700000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': 0},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('29.9800000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('34.4500000000000'),
  'FINANCING': 0,
  'INVESTING': 0},
 'financing': {'GROUP_CFS_FIN_ISSUING_EQUITY': {'desc

All Statements in a Single Call

In [38]:
fin_digest = ledger_model.digest_financial_statements(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

statement_data = fin_digest.get_financial_statements_data()

In [39]:
statement_data['balance_sheet']

{'assets': {'total_balance': Decimal('4.4700000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('ceca82da-05a2-4f09-8033-e1da9508e111'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'op',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('34.4500000000000'),
      'balance_abs': Decimal('34.4500000000000')}],
    'total_balance': Decimal('34.4500000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_inv': {'accounts': [{'account_uuid': UUID('213e4ae3-2603-4957-ac28-4596dff79716'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'op',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
   

In [40]:
statement_data['income_statement']

{'operating': {'revenues': [{'account_uuid': UUID('13d064e4-365c-46aa-b480-2a4c8f5a79ae'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': 'op',
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('34.4500000000000'),
    'balance_abs': Decimal('34.4500000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('c6705d64-4c39-483c-9be8-9dd989a9db0e'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': 'op',
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-29.9800000000000'),
    'balance_abs':

In [41]:
statement_data['cash_flow_statement']

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('4.4700000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': 0},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('29.9800000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('34.4500000000000'),
  'FINANCING': 0,
  'INVESTING': 0},
 'financing': {'GROUP_CFS_FIN_ISSUING_EQUITY': {'desc

# Customers

## Get Customers

In [42]:
customer_qs = entity_model.get_customers()
pd.DataFrame(customer_qs.values()) if PANDAS_INSTALLED else customer_qs

<CustomerModelQueryset [<CustomerModel: C-0000000001: James Davidson>, <CustomerModel: C-0000000002: Dr. Shelby Davis>, <CustomerModel: C-0000000003: Carroll-Garcia>, <CustomerModel: C-0000000004: Ashley Stark>, <CustomerModel: C-0000000005: Brett Swanson>, <CustomerModel: C-0000000006: Melissa Brown>, <CustomerModel: C-0000000007: Ross LLC>, <CustomerModel: C-0000000008: Amanda Espinoza>, <CustomerModel: C-0000000009: Santos-Baker>, <CustomerModel: C-0000000010: Jason Russell>, <CustomerModel: C-0000000011: Christopher Hart>, <CustomerModel: C-0000000012: Davis, Mcdaniel and Parker>, <CustomerModel: C-0000000013: Dorothy Turner>, <CustomerModel: C-0000000014: Pruitt, Vargas and Mosley>, <CustomerModel: C-0000000015: Rachel Young>, <CustomerModel: C-0000000016: Marilyn Hicks>]>

## Create Customers

In [43]:
customer_model = entity_model.create_customer(customer_model_kwargs={
    'customer_name': 'Mr. Big',
    'description': 'A great paying customer!',
})

# Vendors

## Get Vendors

In [44]:
vendor_qs = entity_model.get_vendors()
pd.DataFrame(vendor_qs.values()) if PANDAS_INSTALLED else vendor_qs

<VendorModelQuerySet [<VendorModel: V-0000000001: Ashley Alexander>, <VendorModel: V-0000000002: Ann Griffin>, <VendorModel: V-0000000003: Castillo-Roberts>, <VendorModel: V-0000000004: Wilkins, Patrick and Andrews>, <VendorModel: V-0000000005: Cummings-Johnson>, <VendorModel: V-0000000006: Allison Davis>, <VendorModel: V-0000000007: Phillips, Murphy and Howard>, <VendorModel: V-0000000008: Huff and Sons>, <VendorModel: V-0000000009: Crane-Berry>, <VendorModel: V-0000000010: Simpson LLC>, <VendorModel: V-0000000011: Lynn Barrett DVM>, <VendorModel: V-0000000012: Whitney Lewis>, <VendorModel: V-0000000013: Lyons-Stein>, <VendorModel: V-0000000014: Wiggins Group>, <VendorModel: V-0000000015: Evan Fuentes>, <VendorModel: V-0000000016: Roberts, Ballard and Gray>, <VendorModel: V-0000000017: Lopez, Olson and Frank>, <VendorModel: V-0000000018: John Macias>, <VendorModel: V-0000000019: Ellis, Rose and Davis>, <VendorModel: V-0000000020: Sanchez and Sons>]>

## Create Vendor

In [45]:
vendor_model = entity_model.create_vendor(vendor_model_kwargs={
    'vendor_name': 'ACME LLC',
    'description': 'A Reliable Vendor!'
})

# Invoices

## Get Invoices

In [46]:
invoices_qs = entity_model.get_invoices()
pd.DataFrame(invoices_qs.values()) if PANDAS_INSTALLED else invoices_qs

<InvoiceModelQuerySet [<InvoiceModel: Invoice: I-2023-0000000013 | Draft>, <InvoiceModel: Invoice: I-2023-0000000012 | In Review>, <InvoiceModel: Invoice: I-2022-0000000012 | Draft>, <InvoiceModel: Invoice: I-2023-0000000011 | Draft>, <InvoiceModel: Invoice: I-2023-0000000010 | Void>, <InvoiceModel: Invoice: I-2022-0000000011 | Approved>, <InvoiceModel: Invoice: I-2023-0000000009 | In Review>, <InvoiceModel: Invoice: I-2022-0000000010 | Canceled>, <InvoiceModel: Invoice: I-2022-0000000009 | Approved>, <InvoiceModel: Invoice: I-2023-0000000008 | Draft>, <InvoiceModel: Invoice: I-2022-0000000008 | Draft>, <InvoiceModel: Invoice: I-2023-0000000007 | In Review>, <InvoiceModel: Invoice: I-2022-0000000007 | Draft>, <InvoiceModel: Invoice: I-2022-0000000006 | Paid>, <InvoiceModel: Invoice: I-2023-0000000006 | Draft>, <InvoiceModel: Invoice: I-2023-0000000005 | Paid>, <InvoiceModel: Invoice: I-2023-0000000004 | Paid>, <InvoiceModel: Invoice: I-2022-0000000005 | In Review>, <InvoiceModel: Invoi

## Create Invoice

In [47]:
invoice_model = entity_model.create_invoice(
    customer_model='C-0000000006',
    terms=InvoiceModel.TERMS_NET_30
)

In [48]:
invoice_model

<InvoiceModel: Invoice: I-2025-0000000001 | Draft>

## Add Items to Invoices

In [49]:
invoices_item_models = invoice_model.get_item_model_qs()

# K= number of items...
K = 6

invoice_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(invoices_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
invoice_itemtxs = invoice_model.migrate_itemtxs(itemtxs=invoice_itemtxs,
                                                commit=True,
                                                operation=InvoiceModel.ITEMIZE_REPLACE)
invoice_itemtxs

[<ItemTransactionModel: Invoice Model: 4baf7ab8-df51-4e35-9f9a-71c5ca80569a | 37.80>,
 <ItemTransactionModel: Invoice Model: 4baf7ab8-df51-4e35-9f9a-71c5ca80569a | 90.27>,
 <ItemTransactionModel: Invoice Model: 4baf7ab8-df51-4e35-9f9a-71c5ca80569a | 0.57>,
 <ItemTransactionModel: Invoice Model: 4baf7ab8-df51-4e35-9f9a-71c5ca80569a | 197.34>,
 <ItemTransactionModel: Invoice Model: 4baf7ab8-df51-4e35-9f9a-71c5ca80569a | 303.65>]

In [50]:
invoice_model.amount_due

Decimal('629.63')

# Bills

## Get Bills

In [51]:
bills_qs = entity_model.get_bills()
pd.DataFrame(bills_qs.values()) if PANDAS_INSTALLED else bills_qs

<BillModelQuerySet [<BillModel: Bill: B-2023-0000000018 | Draft>, <BillModel: Bill: B-2023-0000000017 | In Review>, <BillModel: Bill: B-2023-0000000016 | Draft>, <BillModel: Bill: B-2022-0000000017 | Approved>, <BillModel: Bill: B-2022-0000000016 | In Review>, <BillModel: Bill: B-2022-0000000015 | In Review>, <BillModel: Bill: B-2022-0000000014 | In Review>, <BillModel: Bill: B-2022-0000000013 | Paid>, <BillModel: Bill: B-2023-0000000015 | Draft>, <BillModel: Bill: B-2023-0000000014 | Draft>, <BillModel: Bill: B-2023-0000000013 | In Review>, <BillModel: Bill: B-2022-0000000012 | Approved>, <BillModel: Bill: B-2022-0000000011 | In Review>, <BillModel: Bill: B-2023-0000000012 | Draft>, <BillModel: Bill: B-2022-0000000010 | Paid>, <BillModel: Bill: B-2023-0000000011 | In Review>, <BillModel: Bill: B-2022-0000000009 | Draft>, <BillModel: Bill: B-2022-0000000008 | In Review>, <BillModel: Bill: B-2022-0000000007 | Approved>, <BillModel: Bill: B-2023-0000000010 | Paid>, '...(remaining element

## Create Bill

In [52]:
bill_model = entity_model.create_bill(
    vendor_model='V-0000000002',
    terms=BillModel.TERMS_NET_60
)

In [53]:
bill_model

<BillModel: Bill: B-2025-0000000001 | Draft>

## Add Items to Bills

In [54]:
bill_item_models = bill_model.get_item_model_qs()

K = 6

bill_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(bill_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
bill_itemtxs = bill_model.migrate_itemtxs(itemtxs=bill_itemtxs,
                                          commit=True,
                                          operation=BillModel.ITEMIZE_REPLACE)

bill_itemtxs

[<ItemTransactionModel: Bill Model: 62f4b6ff-8975-4ed2-a811-b982c3053806 | 591.67>,
 <ItemTransactionModel: Bill Model: 62f4b6ff-8975-4ed2-a811-b982c3053806 | 241.72>,
 <ItemTransactionModel: Bill Model: 62f4b6ff-8975-4ed2-a811-b982c3053806 | 109.63>,
 <ItemTransactionModel: Bill Model: 62f4b6ff-8975-4ed2-a811-b982c3053806 | 248.61>,
 <ItemTransactionModel: Bill Model: 62f4b6ff-8975-4ed2-a811-b982c3053806 | 355.91>]

In [55]:
bill_model.amount_due

Decimal('1547.54')

# Purchase Orders

## Get Purchase Orders

In [56]:
purchase_orders_qs = entity_model.get_purchase_orders()
pd.DataFrame(purchase_orders_qs.values()) if PANDAS_INSTALLED else purchase_orders_qs

<PurchaseOrderModelQuerySet [<PurchaseOrderModel: PO Model: PO-2023-0000000001 | In Review>, <PurchaseOrderModel: PO Model: PO-2022-0000000001 | Draft>, <PurchaseOrderModel: PO Model: PO-2022-0000000002 | Approved>, <PurchaseOrderModel: PO Model: PO-2023-0000000002 | Approved>, <PurchaseOrderModel: PO Model: PO-2022-0000000003 | Approved>, <PurchaseOrderModel: PO Model: PO-2022-0000000004 | Fulfilled>, <PurchaseOrderModel: PO Model: PO-2023-0000000003 | Approved>, <PurchaseOrderModel: PO Model: PO-2022-0000000005 | In Review>, <PurchaseOrderModel: PO Model: PO-2023-0000000004 | Approved>, <PurchaseOrderModel: PO Model: PO-2023-0000000005 | Approved>, <PurchaseOrderModel: PO Model: PO-2022-0000000006 | Draft>, <PurchaseOrderModel: PO Model: PO-2022-0000000007 | Approved>, <PurchaseOrderModel: PO Model: PO-2023-0000000006 | In Review>, <PurchaseOrderModel: PO Model: PO-2022-0000000008 | Draft>, <PurchaseOrderModel: PO Model: PO-2023-0000000007 | In Review>, <PurchaseOrderModel: PO Model:

## Create Purchase Order

In [57]:
po_model = entity_model.create_purchase_order()

## Add Items to Purchase Orders

In [58]:
po_item_models = po_model.get_item_model_qs()

K = 6

po_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(po_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
po_itemtxs = po_model.migrate_itemtxs(itemtxs=po_itemtxs,
                                      commit=True,
                                      operation=EstimateModel.ITEMIZE_REPLACE)

po_itemtxs

[<ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 88.73>,
 <ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 80.19>,
 <ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 412.49>,
 <ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 131.70>,
 <ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 120.45>,
 <ItemTransactionModel: PO Model: c17659b3-36c4-4a2d-894d-2d9759ea368d | Not Ordered | 248.15>]

In [59]:
po_model.po_amount

Decimal('1081.71')

# Estimates/Contracts

## Get Estimates/Contracts

In [60]:
estimates_qs = entity_model.get_estimates()
pd.DataFrame(estimates_qs.values()) if PANDAS_INSTALLED else estimates_qs

<EstimateModelQuerySet [<EstimateModel: Estimate E-2025-0000000025 | Customer Estimate 2023-01-21 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000024 | Customer Estimate 2023-03-01 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000023 | Customer Estimate 2022-10-08 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000022 | Customer Estimate 2022-10-25 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000021 | Customer Estimate 2022-10-25 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000020 | Customer Estimate 2022-11-21 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000019 | Customer Estimate 2023-02-21 00:00:00+00:00>, <EstimateModel: Contract E-2025-0000000018 | Customer Estimate 2023-03-19 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000017 | Customer Estimate 2023-03-13 00:00:00+00:00>, <EstimateModel: Contract E-2025-0000000016 | Customer Estimate 2022-11-07 00:00:00+00:00>, <EstimateModel: Estimate E-2025-0000000015 | Customer Estimate 202

## Create Estimate

In [61]:
estimate_model = entity_model.create_estimate(
    estimate_title='A quote for new potential customer!',
    customer_model='C-0000000009',
    contract_terms=EstimateModel.CONTRACT_TERMS_FIXED
)

## Add Items to Estimates

In [62]:
estimate_item_models = estimate_model.get_item_model_qs()

K = 6

estimate_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'unit_revenue': round(random() * 20, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(estimate_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
estimate_itemtxs = estimate_model.migrate_itemtxs(itemtxs=estimate_itemtxs,
                                                  commit=True,
                                                  operation=EstimateModel.ITEMIZE_REPLACE)

estimate_itemtxs

[<ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 121.52>,
 <ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 79.82>,
 <ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 1.15>,
 <ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 322.76>,
 <ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 613.22>,
 <ItemTransactionModel: Estimate/Contract Model: 46a4ae9d-263a-45b3-b4af-68de7493e929 | 115.60>]

In [63]:
estimate_model.get_cost_estimate()

Decimal('1254.07')

In [64]:
estimate_model.get_revenue_estimate()

Decimal('1565.297500000000007869260799')

In [65]:
estimate_model.get_profit_estimate()

Decimal('311.227500000000007869260799')

In [66]:
estimate_model.get_gross_margin_estimate(as_percent=True)

24.817394563301896

# Bank Accounts

## Get Bank Accounts

In [67]:
bank_accounts_qs = entity_model.get_bank_accounts()
pd.DataFrame(bank_accounts_qs.values()) if PANDAS_INSTALLED else bank_accounts_qs

<BankAccountModelQuerySet [<BankAccountModel: Checking Bank Account: One Big Company, LLC Checking Account>, <BankAccountModel: Savings Bank Account: One Big Company, LLC Savings Account>]>

## Create Bank Account

In [68]:
bank_account_model = entity_model.create_bank_account(name='A big bank account!',
                                                      account_type='checking')

# Items

## Unit of Measures

### Get Unit of Measures

In [69]:
uom_qs = entity_model.get_uom_all()
pd.DataFrame(uom_qs.values()) if PANDAS_INSTALLED else uom_qs

<UnitOfMeasureModelQuerySet [<UnitOfMeasureModel: Pound (lb)>, <UnitOfMeasureModel: Linear Feet (ln-ft)>, <UnitOfMeasureModel: Man Hour (man-hour)>, <UnitOfMeasureModel: Pallet (pallet)>, <UnitOfMeasureModel: Square Fee t (sq-ft)>, <UnitOfMeasureModel: Unit (unit)>]>

### Create a UOM

In [70]:
uom_model_ft = entity_model.create_uom(
    name='Linear Feet',
    unit_abbr='lin-ft'
)

### Get Some UoMs

In [71]:
uom_model_unit = uom_qs.get(unit_abbr__exact='unit')
uom_model_man_hr = uom_qs.get(unit_abbr__exact='man-hour')

## Expenses

### Get Expense Items

In [72]:
expenses_qs = entity_model.get_items_expenses()
pd.DataFrame(expenses_qs.values()) if PANDAS_INSTALLED else expenses_qs

<ItemModelQuerySet [<ItemModel: Expense: Expense Item 2730 | Equipment>, <ItemModel: Expense: Expense Item 2307 | Material>, <ItemModel: Expense: Expense Item 3752 | Equipment>, <ItemModel: Expense: Expense Item 2014 | Material>, <ItemModel: Expense: Expense Item 3401 | Lump Sum>, <ItemModel: Expense: Expense Item 7428 | Labor>, <ItemModel: Expense: Expense Item 3588 | Labor>, <ItemModel: Expense: Expense Item 7555 | Lump Sum>, <ItemModel: Expense: Expense Item 4180 | Material>, <ItemModel: Expense: Expense Item 1575 | Material>, <ItemModel: Expense: Expense Item 4173 | Equipment>, <ItemModel: Expense: Expense Item 7986 | Equipment>, <ItemModel: Expense: Expense Item 4257 | Material>, <ItemModel: Expense: Expense Item 8859 | Material>, <ItemModel: Expense: Expense Item 3029 | Other>, <ItemModel: Expense: Expense Item 4034 | Material>, <ItemModel: Expense: Expense Item 8195 | Other>, <ItemModel: Expense: Expense Item 2601 | Lump Sum>, <ItemModel: Expense: Expense Item 1334 | Lump Sum>, 

### Create Expense Item

In [73]:
expense_item_model = entity_model.create_item_expense(
    name='Premium Pencils',
    uom_model=uom_model_unit,
    expense_type=ItemModel.ITEM_TYPE_MATERIAL
)

In [74]:
expense_item_model.is_expense()

True

## Services

### Get Service Items

In [75]:
services_qs = entity_model.get_items_services()
pd.DataFrame(services_qs.values()) if PANDAS_INSTALLED else services_qs

<ItemModelQuerySet [<ItemModel: Service: Service #7777 | Labor>, <ItemModel: Service: Service #5722 | Labor>, <ItemModel: Service: Service #8867 | Labor>, <ItemModel: Service: Service #7816 | Labor>, <ItemModel: Service: Service #8998 | Labor>, <ItemModel: Service: Service #7712 | Labor>, <ItemModel: Service: Service #7535 | Labor>, <ItemModel: Service: Service #9844 | Labor>, <ItemModel: Service: Service #4243 | Labor>, <ItemModel: Service: Service #3464 | Labor>, <ItemModel: Service: Service #5665 | Labor>, <ItemModel: Service: Service #3873 | Labor>, <ItemModel: Service: Service #4163 | Labor>, <ItemModel: Service: Service #2979 | Labor>, <ItemModel: Service: Service #6268 | Labor>, <ItemModel: Service: Service #7352 | Labor>, <ItemModel: Service: Service #6987 | Labor>, <ItemModel: Service: Service #7061 | Labor>, <ItemModel: Service: Service #4886 | Labor>, <ItemModel: Service: Service #9026 | Labor>, '...(remaining elements truncated)...']>

### Create Service Item

In [76]:
service_model = entity_model.create_item_service(
    name='Yoga Class',
    uom_model=uom_model_man_hr
)

In [77]:
service_model.is_service()

True

## Products

### Get Product Items

In [78]:
products_qs = entity_model.get_items_products()
pd.DataFrame(products_qs.values()) if PANDAS_INSTALLED else products_qs

<ItemModelQuerySet [<ItemModel: Product: Product #3237>, <ItemModel: Product: Product #6044>, <ItemModel: Product: Product #8888>, <ItemModel: Product: Product #7050>, <ItemModel: Product: Product #6920>, <ItemModel: Product: Product #6418>, <ItemModel: Product: Product #2290>, <ItemModel: Product: Product #1736>, <ItemModel: Product: Product #7452>, <ItemModel: Product: Product #4772>, <ItemModel: Product: Product #3127>, <ItemModel: Product: Product #3228>, <ItemModel: Product: Product #3228>, <ItemModel: Product: Product #9034>, <ItemModel: Product: Product #3234>, <ItemModel: Product: Product #5255>, <ItemModel: Product: Product #6762>, <ItemModel: Product: Product #7904>, <ItemModel: Product: Product #7365>, <ItemModel: Product: Product #8815>, '...(remaining elements truncated)...']>

### Create Product Items

In [79]:
product_model = entity_model.create_item_product(
    name='1/2" Premium PVC Pipe',
    uom_model=uom_model_ft,
    item_type=ItemModel.ITEM_TYPE_MATERIAL
)

In [80]:
product_model.is_product()

True

## Inventory

### Get Inventory Items

In [81]:
inventory_qs = entity_model.get_items_inventory()
pd.DataFrame(inventory_qs.values()) if PANDAS_INSTALLED else inventory_qs

<ItemModelQuerySet [<ItemModel: Product: Product #3237>, <ItemModel: Product: Product #6044>, <ItemModel: Product: Product #8888>, <ItemModel: Product: Product #7050>, <ItemModel: Product: Product #6920>, <ItemModel: Product: Product #6418>, <ItemModel: Product: Product #2290>, <ItemModel: Product: Product #1736>, <ItemModel: Product: Product #7452>, <ItemModel: Product: Product #4772>, <ItemModel: Product: Product #3127>, <ItemModel: Product: Product #3228>, <ItemModel: Product: Product #3228>, <ItemModel: Product: Product #9034>, <ItemModel: Product: Product #3234>, <ItemModel: Product: Product #5255>, <ItemModel: Product: Product #6762>, <ItemModel: Product: Product #7904>, <ItemModel: Product: Product #7365>, <ItemModel: Product: Product #8815>, '...(remaining elements truncated)...']>

### Create Inventory Items

In [82]:
inventory_model = entity_model.create_item_inventory(
    name='A Home to Flip!',
    uom_model=uom_model_unit,
    item_type=ItemModel.ITEM_TYPE_LUMP_SUM
)

In [83]:
inventory_model.is_inventory()

True

# Financial Statement PDF Reports

## Set Up
- Must enable PDF support by installing dependencies via *pipenv*.
    - pipenv install --categories pdf

## Balance Sheet

In [84]:
bs_report = entity_model.get_balance_sheet_statement(
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...
# will raise not implemented error if PDF support is not enabled...

### Balance Sheet Statement Raw Data

In [85]:
bs_report.get_report_data()

{'assets': {'total_balance': Decimal('50125.080000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('ceca82da-05a2-4f09-8033-e1da9508e111'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('49810.32000000000'),
      'balance_abs': Decimal('49810.32000000000')}],
    'total_balance': Decimal('49810.32000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_recv': {'accounts': [{'account_uuid': UUID('d58be633-4976-4292-b550-f1a686ab4da2'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'asset

## Income Statement

In [86]:
ic_report = entity_model.get_income_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...
# will raise not implemented error if PDF support is not enabled...

### Income Statement Raw Data

In [87]:
ic_report.get_report_data()

{'operating': {'revenues': [{'account_uuid': UUID('13d064e4-365c-46aa-b480-2a4c8f5a79ae'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('1543.03000000000'),
    'balance_abs': Decimal('1543.03000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('c6705d64-4c39-483c-9be8-9dd989a9db0e'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-396.490000000000'),
    'balance_abs':

## Cash Flow Statement

In [88]:
cf_report = entity_model.get_cash_flow_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...

### Cash Flow Statement Raw Data

In [89]:
cf_report.get_report_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('-6137.46000000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-711.250000000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('396.490000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('6262.54000000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('-189.68000000000000'),
  'FINANCING': Decimal('48978

## All Financial Statements Data in a single Call

In [90]:
reports = entity_model.get_financial_statements(
    user_model=user_model,
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...

/Users/elarroba/PycharmProjects/django-ledger/notebooks


In [91]:
reports.balance_sheet_statement.get_report_data()

{'assets': {'total_balance': Decimal('50125.080000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('ceca82da-05a2-4f09-8033-e1da9508e111'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'fin_equity',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('50000'),
      'balance_abs': Decimal('50000')},
     {'account_uuid': UUID('ceca82da-05a2-4f09-8033-e1da9508e111'),
      'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'fin_std',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': 

In [92]:
reports.income_statement.get_report_data()

{'operating': {'revenues': [{'account_uuid': UUID('13d064e4-365c-46aa-b480-2a4c8f5a79ae'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('1543.03000000000'),
    'balance_abs': Decimal('1543.03000000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('c6705d64-4c39-483c-9be8-9dd989a9db0e'),
    'coa_slug': 'coa-9adzs-a7sjgs81fkl9sm4',
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-396.490000000000'),
    'balance_abs':

In [93]:
reports.cash_flow_statement.get_report_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('-6137.46000000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-711.250000000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('396.490000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('6262.54000000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('-189.68000000000000'),
  'FINANCING': Decimal('48978